# Assignment 2: Bryant Parchinski

## Imports

In [27]:
import pandas as pd
import numpy as np

print("Imports setup! :)")

Imports setup! :)


## Data setup

In [28]:
data_fp = "./2000_acs_sample.dta"

df = pd.read_stata(data_fp)

print("DataFrame loaded! :)")
print(f"Shape of DataFrame: {df.shape[0]} rows, {df.shape[1]} columns")

DataFrame loaded! :)
Shape of DataFrame: 28172 rows, 16 columns


## First inspection of names and head rows

In [29]:
print("Initial column names:")
print(df.columns.tolist())

print("\nHead of Loaded Data:")
display(df.head())

Initial column names:
['year', 'datanum', 'serial', 'hhwt', 'gq', 'us2000c_serialno', 'pernum', 'perwt', 'us2000c_pnum', 'us2000c_sex', 'us2000c_age', 'us2000c_hispan', 'us2000c_race1', 'us2000c_marstat', 'us2000c_educ', 'us2000c_inctot']

Head of Loaded Data:


,year,datanum,serial,hhwt,gq,us2000c_serialno,pernum,perwt,us2000c_pnum,us2000c_sex,us2000c_age,us2000c_hispan,us2000c_race1,us2000c_marstat,us2000c_educ,us2000c_inctot
0,2000,4,37.0,100.0,Households under 1970 definition,0365663,1,100.0,01,2,20,01,1,5,11,0010000
1,2000,4,37.0,100.0,Households under 1970 definition,0365663,2,100.0,02,2,19,01,1,5,11,0005300
2,2000,4,37.0,100.0,Households under 1970 definition,0365663,3,100.0,03,2,19,01,2,5,11,0004700
3,2000,4,241.0,100.0,Households under 1970 definition,2894822,1,100.0,01,2,50,01,1,5,14,0032500
4,2000,4,242.0,100.0,Households under 1970 definition,2896802,1,100.0,01,2,29,01,1,5,13,0030000


### Identify and drop columns with all the same value

In [30]:
single_value_cols = []
for col in df.columns:
    unique_vals = df[col].unique()
    if len(unique_vals) == 1:
        single_value_cols.append(col)

print("Columns that have only one value:", single_value_cols)

# drop the specialy requested cols
columns_to_drop = single_value_cols + ['us2000c_pnum', 'us2000c_serialno']

df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

print("Unwanted columns dropped successfully. Yipee! :)")
print(f"Remaining columns ({len(df.columns)}): {df.columns.tolist()}")

Columns that have only one value: ['year', 'datanum', 'hhwt', 'perwt']
Unwanted columns dropped successfully. Yipee! :)
Remaining columns (10): ['serial', 'gq', 'pernum', 'us2000c_sex', 'us2000c_age', 'us2000c_hispan', 'us2000c_race1', 'us2000c_marstat', 'us2000c_educ', 'us2000c_inctot']


### Rename columns to specified names

In [31]:
rename_map = {
    "serial": "household",
    "pernum": "person",
    "us2000c_sex": "sex",
    "us2000c_age": "age",
    "us2000c_hispan": "hispanic",
    "us2000c_race1": "race",
    "us2000c_marstat": "marital_status",
    "us2000c_educ": "edu",
    "us2000c_inctot": "income"
}

df.rename(columns=rename_map, inplace=True)

print("Columns after that smooth rename:")
print(df.columns.tolist())

Columns after that smooth rename:
['household', 'gq', 'person', 'sex', 'age', 'hispanic', 'race', 'marital_status', 'edu', 'income']


### Checking DataFrame stucture after cleaning

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28172 entries, 0 to 28171
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   household       28172 non-null  float64 
 1   gq              28172 non-null  category
 2   person          28172 non-null  int16   
 3   sex             28172 non-null  object  
 4   age             28172 non-null  object  
 5   hispanic        28172 non-null  object  
 6   race            28172 non-null  object  
 7   marital_status  28172 non-null  object  
 8   edu             28172 non-null  object  
 9   income          28172 non-null  object  
dtypes: category(1), float64(1), int16(1), object(7)
memory usage: 1.8+ MB


### Convert income to numeric following the instructions then check income type to ensure success

In [33]:
if 'income' in df.columns and df['income'].dtype == object:
    df['income'] = pd.to_numeric(df['income'], errors='coerce')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28172 entries, 0 to 28171
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   household       28172 non-null  float64 
 1   gq              28172 non-null  category
 2   person          28172 non-null  int16   
 3   sex             28172 non-null  object  
 4   age             28172 non-null  object  
 5   hispanic        28172 non-null  object  
 6   race            28172 non-null  object  
 7   marital_status  28172 non-null  object  
 8   edu             28172 non-null  object  
 9   income          22015 non-null  float64 
dtypes: category(1), float64(2), int16(1), object(6)
memory usage: 1.8+ MB


### Replace numeric codes for 'sex' and 'marital_status'

In [34]:
if 'sex' in df.columns and pd.api.types.is_float_dtype(df['sex']):
    df['sex'] = df['sex'].astype('Int64')

if 'marital_status' in df.columns and pd.api.types.is_float_dtype(df['marital_status']):
    df['marital_status'] = df['marital_status'].astype('Int64')

sex_map = {
    1: "Male",
    2: "Female",
    9: "Missing"
}

marital_map = {
    1: "Married",
    2: "Widowed",
    3: "Divorced",
    4: "Separated",
    5: "Never Married",
    6: "Missing/Unknown",
    9: "Missing/Unknown"
}

if 'sex' in df.columns:
    df['sex'] = df['sex'].map(sex_map)

if 'marital_status' in df.columns:
    df['marital_status'] = df['marital_status'].map(marital_map)

### Replace missing incomes with the mode Income

In [35]:
if 'income' in df.columns:
    mode_val = df['income'].mode(dropna=True)
    if not mode_val.empty:
        df['income'].fillna(mode_val[0], inplace=True)

/tmp/ipykernel_1652/3575119999.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['income'].fillna(mode_val[0], inplace=True)


In [36]:
df.info()
display(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28172 entries, 0 to 28171
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   household       28172 non-null  float64 
 1   gq              28172 non-null  category
 2   person          28172 non-null  int16   
 3   sex             0 non-null      object  
 4   age             28172 non-null  object  
 5   hispanic        28172 non-null  object  
 6   race            28172 non-null  object  
 7   marital_status  0 non-null      object  
 8   edu             28172 non-null  object  
 9   income          28172 non-null  float64 
dtypes: category(1), float64(2), int16(1), object(6)
memory usage: 1.8+ MB


,household,gq,person,sex,age,hispanic,race,marital_status,edu,income
0,37.0,Households under 1970 definition,1,NaN,20,01,1,NaN,11,10000.0
1,37.0,Households under 1970 definition,2,NaN,19,01,1,NaN,11,5300.0
2,37.0,Households under 1970 definition,3,NaN,19,01,2,NaN,11,4700.0
3,241.0,Households under 1970 definition,1,NaN,50,01,1,NaN,14,32500.0
4,242.0,Households under 1970 definition,1,NaN,29,01,1,NaN,13,30000.0
5,296.0,Other group quarters,1,NaN,20,01,6,NaN,09,3000.0
6,377.0,Households under 1970 definition,1,NaN,69,01,1,NaN,01,51900.0
7,418.0,Households under 1970 definition,1,NaN,59,01,1,NaN,08,12200.0
8,465.0,Households under 1970 definition,1,NaN,55,01,2,NaN,12,0.0
9,465.0,Households under 1970 definition,2,NaN,47,01,2,NaN,01,2600.0
